In [ ]:
%matplotlib inline

In [ ]:
from urllib.request import urlopen
from PIL import Image
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [ ]:
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, RandomBrightnessContrast, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, Flip, OneOf, Compose, RGBShift, Cutout, Resize
)

In [ ]:
def read_image(image_id, box):
    image = Image.open('../input/train_images/{}.jpg'.format(image_id))
    x, y, w, h = box
    # Crop as Character's PIL Image
    char_img = image.crop((x, y, x+w, y+h))
#     # Gray-Scale Character's PIL Image where the channel is 1
#     char_img = char_img.convert('L')
#     # Resize Character's PIL Image
#     char_img = char_img.resize((48, 48))
    char_img = np.asarray(char_img)
    return char_img

In [ ]:
def augment_flips_color(p=.5):
    return Compose([
        RGBShift(r_shift_limit=100, g_shift_limit=100, b_shift_limit=100, p=p),
        ShiftScaleRotate(shift_limit=0.1, rotate_limit=10, p=p),
        Cutout(num_holes=3, max_h_size=5, max_w_size=5, fill_value=0, p=p),
        Resize(48, 48, p=1)
    ], p=p)

In [ ]:
def augment(aug, image):
    image = aug(image=image)['image']
    return image

In [ ]:
def show_image(image):
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    plt.show()

In [ ]:
image = read_image("100241706_00004_2", (1231, 3465, 133, 53))
print(image.shape)
show_image(image)

In [ ]:
aug = augment_flips_color(p=1)
image = augment(aug, image)
print(image.shape)
show_image(image)

## Demo

In [ ]:
def download_image(url):
    data = urlopen(url).read()
    data = np.frombuffer(data, np.uint8)
    image = cv2.imdecode(data, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

In [ ]:
image = download_image('https://d177hi9zlsijyy.cloudfront.net/wp-content/uploads/sites/2/2018/05/11202041/180511105900-atlas-boston-dynamics-robot-running-super-tease.jpg')

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.show()

In [ ]:
print(type(image))

In [ ]:
def augment_and_show(aug, image):
    image = aug(image=image)['image']
    plt.figure(figsize=(10, 10))
    plt.imshow(image)

In [ ]:
aug = RGBShift(r_shift_limit=100, g_shift_limit=100, b_shift_limit=100, p=1)
augment_and_show(aug, image)

In [ ]:
aug = Cutout(num_holes=10, max_h_size=30, max_w_size=30, fill_value=0, p=1)
augment_and_show(aug, image)

In [ ]:
aug = ShiftScaleRotate(shift_limit=0.1, rotate_limit=10, p=1)
augment_and_show(aug, image)

In [ ]:
def augment_flips_color(p=.5):
    return Compose([
        RGBShift(r_shift_limit=100, g_shift_limit=100, b_shift_limit=100, p=p),
        ShiftScaleRotate(shift_limit=0.1, rotate_limit=10, p=p),
        Cutout(num_holes=3, max_h_size=5, max_w_size=5, fill_value=0, p=p),
        Resize(48, 48, p=1)
    ], p=p)

In [ ]:
aug = augment_flips_color(p=1)
augment_and_show(aug, image)

In [ ]:
def strong_aug(p=.5):
    return Compose([
        RandomRotate90(),
        Flip(),
        Transpose(),
        OneOf([
            IAAAdditiveGaussianNoise(),
            GaussNoise(),
        ], p=0.2),
        OneOf([
            MotionBlur(p=.2),
            MedianBlur(blur_limit=3, p=0.1),
            Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
        OneOf([
            OpticalDistortion(p=0.3),
            GridDistortion(p=.1),
            IAAPiecewiseAffine(p=0.3),
        ], p=0.2),
        OneOf([
            CLAHE(clip_limit=2),
            IAASharpen(),
            IAAEmboss(),
            RandomBrightnessContrast(),            
        ], p=0.3),
        HueSaturationValue(p=0.3),
    ], p=p)

In [ ]:
aug = strong_aug(p=1)
augment_and_show(aug, image)